In [1]:
import numpy as np
import pandas as pd

In [2]:
df_abb = pd.read_csv('./data/state-abbrevs.csv')
df_area = pd.read_csv('./data/state-areas.csv')
df_popu = pd.read_csv('./data/state-population.csv')

In [3]:
df_abb.head(2)

,state,abbreviation
0,Alabama,AL
1,Alaska,AK


In [4]:
df_area.head(2)

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425


In [5]:
df_popu.head(2)

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0


## 将人口数据df_popu和各州简称df_abb 合并, how='outer', 防止数据丢失

In [58]:
df_1 = pd.merge(df_abb, df_popu, left_on='abbreviation',right_on='state/region',how='outer')
# 删除重复的column
df_1.drop(columns=['abbreviation'],inplace=True)

In [59]:
# df_1.drop(labels='abbreviation',axis=1, inplace=True)

In [60]:
df_1.head()

,state,state/region,ages,year,population
0,Alabama,AL,under18,2012,1117489.0
1,Alabama,AL,total,2012,4817528.0
2,Alabama,AL,under18,2010,1130966.0
3,Alabama,AL,total,2010,4785570.0
4,Alabama,AL,under18,2011,1125763.0


## 查看存在缺失数据的列

In [61]:
# 方式 1
df_1.isnull().sum()

state           96
state/region     0
ages             0
year             0
population      20
dtype: int64

In [63]:
# method 2
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2544 entries, 0 to 2543
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   state         2448 non-null   object 
 1   state/region  2544 non-null   object 
 2   ages          2544 non-null   object 
 3   year          2544 non-null   int64  
 4   population    2524 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 119.2+ KB


In [23]:
df_popu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2544 entries, 0 to 2543
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   state/region  2544 non-null   object 
 1   ages          2544 non-null   object 
 2   year          2544 non-null   int64  
 3   population    2524 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 79.6+ KB


In [24]:
df_area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   state          52 non-null     object
 1   area (sq. mi)  52 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 960.0+ bytes


In [25]:
df_abb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   state         51 non-null     object
 1   abbreviation  51 non-null     object
dtypes: object(2)
memory usage: 944.0+ bytes


## 找到哪些 state/region 使得 state的值为NaN，进行去重操作

In [64]:
df_1.head()

,state,state/region,ages,year,population
0,Alabama,AL,under18,2012,1117489.0
1,Alabama,AL,total,2012,4817528.0
2,Alabama,AL,under18,2010,1130966.0
3,Alabama,AL,total,2010,4785570.0
4,Alabama,AL,under18,2011,1125763.0


In [66]:
# 1. find all nan in df_1['stat']
filt = df_1['state'].isnull()
# 2. select all rows with 'filt'
df_1[filt]
# 3. sllect all 'abbreviation'
df_1[filt]['state/region']
# 4. remove duplicated 'abbreviation'
df_1[filt]['state/region'].unique()

array(['PR', 'USA'], dtype=object)

## 把'state/region'中所有nan，替换为正确的值，去掉这一个column的所有NaN

In [67]:
# 采用fillna(value='xxx')的方式填充nan
# 使用直接付值的方式

In [71]:
# 找出所有为‘USA’的行
filt = df_1['state/region'] == 'USA'
# 找出行中的空值
df_1[filt]['state']
# 找出空值的index
indexs = df_1[filt]['state'].index
# 对空值付值
df_1.loc[indexs,'state'] = 'United States'

In [74]:
# PR
filt = df_1['state/region'] == 'PR'
# 找出行中的空值
df_1[filt]['state']
# 找出空值的index
indexs = df_1[filt]['state'].index
# 对空值付值
df_1.loc[indexs,'state'] = 'PPPRRR'

In [75]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2544 entries, 0 to 2543
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   state         2544 non-null   object 
 1   state/region  2544 non-null   object 
 2   ages          2544 non-null   object 
 3   year          2544 non-null   int64  
 4   population    2524 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 183.8+ KB


## 把各州面积合并到 df_1

In [79]:
df_area.head(2)

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425


In [81]:
df_area.shape

(52, 2)

In [80]:
df_1.head(2)

,state,state/region,ages,year,population
0,Alabama,AL,under18,2012,1117489.0
1,Alabama,AL,total,2012,4817528.0


In [82]:
df_1.shape

(2544, 5)

In [95]:
df_abb_pop_area = pd.merge(df_1,df_area,how='outer')

In [96]:
df_abb_pop_area.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2545 entries, 0 to 2544
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   state          2545 non-null   object 
 1   state/region   2544 non-null   object 
 2   ages           2544 non-null   object 
 3   year           2544 non-null   float64
 4   population     2524 non-null   float64
 5   area (sq. mi)  2449 non-null   float64
dtypes: float64(3), object(3)
memory usage: 139.2+ KB


### 找到并删除area缺失的行

In [97]:
# all states with area is null
filt = df_abb_pop_area['area (sq. mi)'].isnull()
df_abb_pop_area[filt]['state']

2448           PPPRRR
2449           PPPRRR
2450           PPPRRR
2451           PPPRRR
2452           PPPRRR
            ...      
2539    United States
2540    United States
2541    United States
2542    United States
2543    United States
Name: state, Length: 96, dtype: object

In [98]:
# 找出index
filt = df_abb_pop_area['area (sq. mi)'].isnull()
indexs = df_abb_pop_area[filt].index
# drop rows
df_abb_pop_area.drop(labels=indexs,axis=0,inplace=True)

In [99]:
df_abb_pop_area.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2449 entries, 0 to 2544
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   state          2449 non-null   object 
 1   state/region   2448 non-null   object 
 2   ages           2448 non-null   object 
 3   year           2448 non-null   float64
 4   population     2448 non-null   float64
 5   area (sq. mi)  2449 non-null   float64
dtypes: float64(3), object(3)
memory usage: 133.9+ KB


## 找出2010年全民人口数据

In [100]:
df_abb_pop_area.head(2)

,state,state/region,ages,year,population,area (sq. mi)
0,Alabama,AL,under18,2012.0,1117489.0,52423.0
1,Alabama,AL,total,2012.0,4817528.0,52423.0


In [104]:
# filter 2010
df_abb_pop_area.query("(year==2010) & (ages=='total')")

,state,state/region,ages,year,population,area (sq. mi)
3,Alabama,AL,total,2010.0,4785570.0,52423.0
91,Alaska,AK,total,2010.0,713868.0,656425.0
101,Arizona,AZ,total,2010.0,6408790.0,114006.0
189,Arkansas,AR,total,2010.0,2922280.0,53182.0
197,California,CA,total,2010.0,37333601.0,163707.0
283,Colorado,CO,total,2010.0,5048196.0,104100.0
293,Connecticut,CT,total,2010.0,3579210.0,5544.0
379,Delaware,DE,total,2010.0,899711.0,1954.0
389,District of Columbia,DC,total,2010.0,605125.0,68.0
475,Florida,FL,total,2010.0,18846054.0,65758.0


### 各州人口密度

In [108]:
df_abb_pop_area['dentity'] = df_abb_pop_area['population'] / df_abb_pop_area['area (sq. mi)']

In [109]:
df_abb_pop_area.head()

,state,state/region,ages,year,population,area (sq. mi),dentity
0,Alabama,AL,under18,2012.0,1117489.0,52423.0,21.316769
1,Alabama,AL,total,2012.0,4817528.0,52423.0,91.897221
2,Alabama,AL,under18,2010.0,1130966.0,52423.0,21.573851
3,Alabama,AL,total,2010.0,4785570.0,52423.0,91.287603
4,Alabama,AL,under18,2011.0,1125763.0,52423.0,21.474601


### 找出人口最密集的state

In [116]:
df_abb_pop_area.sort_values(by=['dentity'],axis=0,ascending=False).iloc[0,0]

'District of Columbia'